# ¿Cúal es el mejor modelo para determinar el plan de un usuario?

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Por lo tanto desarrollaremos un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Para ello analizaremos datasets donde se registra el comportamiento de cada suscriptor, dicho dataset ya se encuentra procesado por lo que nos dedicaremos a solo realizar el modelo. Con ello crearemos un modelo clasificación que escoja el plan correcto.

**Tabla de contenido:**

1. Análisis de datasets
2. Desarrollo del modelo
3. Calidad del modelo
4. Conclusión general

**Objetivos**

1. Desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

**Descripción del dataset del proyecto:**

- `сalls`: número de llamadas
- `minutes`: duración total de la llamada en minutos
- `messages`: número de mensajes de texto
- `mb_used`: tráfico de Internet utilizado en MB
- `is_ultra`: plan para el mes actual (Ultra - 1, Smart - 0)

## Análisis de datasets

In [1]:
# Importar librerías necesarias
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

### Carga de datos

In [2]:
# Importar dataset
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


## Desarrollo del Modelo

Segmentaremos nuestros datos en un conjunto de entrenamiento, uno de validación y uno de prueba. Se establecerán las siguientes proporciones del dataset:

- **Entrenamiento:** 60%
- **Validación:** 20%
- **Prueba:** 20%

### Modelos de Clasifiación

Ahora que tenemos nuestros datos segementados, procederemos a la prueba de distintos modelos con diferentes hiperparámetros. El objetivo es encontrar el mejor modelo, el más preciso. Probaremos con los siguientes modelos:

- **Árbol de decisiones**
- **Bosque aleatorio**
- **Regresión Logística**

Primero definiremos las características que debe evaluar el modelo (features) y el objetivo (target).

In [3]:
# Definir feutures y target del modelo
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Segmentaremos en conjunto de entrenamiento y de prueba según las prociones descritas anteriormente.

In [4]:
# Segmentar el dataset en conjutno de entrenamiento y prueba
features_train, features_vt, target_train, target_vt = train_test_split(
    features, target, test_size=0.40, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_vt, target_vt, test_size=0.50, random_state=12345)

#### Árbol de decisiones

Para optimizar la precisión de nuestro modelo de árbol de decisión, realizaremos iteraciones variando la profundidad máxima del árbol, ajustando así el hiperparámetro max_depth.

In [5]:
# Desarrollar modelo de árbol de decisiones
best_depth = 0
best_score = 0

# Seleccionar el rango del hiperparámetro
for depth in range(1,10):
    
    # Modelo    
    model_three = DecisionTreeClassifier(random_state=12345,  max_depth=depth)
    
    # Entrenar el modelo    
    model_three.fit(features_train, target_train)
    
    # Calcular precisión del modelo    
    score = model_three.score(features_valid, target_valid)
    
    # Encontrar el mejor número de estimadores    
    if score > best_score:
        best_score = score
        best_depth = depth
        
print("Accuracy del mejor modelo en el conjunto de validación (max_depth = {}): {}".format(best_depth, best_score))

Accuracy del mejor modelo en el conjunto de validación (max_depth = 3): 0.7853810264385692


**Nuestro modelo de árbol de decisiones alcanza un precisión de 0.785 utilizando una profundidad de árbol de decisión (max_depth) de 5.**

#### Bosque aleatorio

El siguiente modelo a probar será el bosque aletorio donde iteraremos el modelo cambiando el número de estimadores (n_estimators), es decir el número de árboles del bosque.

In [6]:
# Desarrollar modelo de bosque aleatorio
best_score = 0
best_est = 0
best_depth = 0

# Seleccionar el rango del hiperparámetro
for est in range(1,50): 
    for depth in range(1,10):
    
        # Modelo
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)

        # Entrenar el modelo
        model_forest.fit(features_train, target_train)

        # Calcular precisión del modelo
        score = model_forest.score(features_valid, target_valid)

        # Encontrar el mejor número de estimadores
        if score > best_score:
            best_score = score
            best_est = est
            best_depth = depth
        
print("Accuracy del mejor modelo en el conjunto de validación (n_estimators = {}, max_depth = {}): {}".format(best_est, best_depth, best_score))

Accuracy del mejor modelo en el conjunto de validación (n_estimators = 40, max_depth = 8): 0.8087091757387247


**Nuestro modelo de bosque aleatorio alcanza un precisión de 0.809 utilizando un cantidad de estimadores (n_estimators) de 40 y profundidad  árboles (max_depht) de 8. A pesar que la precisión del modelo es mayor al de árbol de decisión, el modelo es más lento debido a la cantidad estimadores que se deben usar.**

#### Regresión logística

El último modelo a utlizar será la regresión logística.

In [7]:
# Desarrollar modelo de regresión logística
model_logistic = LogisticRegression(random_state=54321, solver='liblinear')

# Entrenar el modelo
model_logistic.fit(features_train, target_train) 

# Calcular precisión del modelo en el conjunto de entrenamiento
score_train = model_logistic.score(features_train, target_train)

# Calcular precisión del modelo en el conjunto de validación
score_valid = model_logistic.score(features_valid, target_valid)

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7505186721991701


**El modelo de regresión logística resulto ser el más bajo en precisión: 0.751.**

### Conclusión y elección de modelo

Luego de probar nuestros tres modelos, pudimos determinar que **el mejor modelo a aplicar en este caso es el del bosque de árboles utilizando 23 árboles (n_estimators) y profundidad de 8 (max_depth).**

## Calidad del Modelo

Ahora probaremos la calidad del modelo comparando la diferencia entre la precisión del modelo con el conjunto de prueba.

In [8]:
# Establecer modelo final
final_model = RandomForestClassifier(random_state=12345, n_estimators=23)
final_model.fit(features_train, target_train)

RandomForestClassifier(n_estimators=23, random_state=12345)

In [9]:
# Desarrollar modelo final para modelo de entrenamiento y modelo de prueba
train_predictions = final_model.predict(features_train)
test_predictions = final_model.predict(features_test)

print('Exactitud:')
print('Training set:', accuracy_score(target_train, train_predictions))
print('Test set:', accuracy_score(target_test, test_predictions))

Exactitud:
Training set: 0.9937759336099585
Test set: 0.7807153965785381


**Podemos ver que la exactitud o precisión de nuestro model final en nuestro conjunto de prueba es de 0.781, la cual se aleja al compararlo con nuestro conjunto de entrenamiento. Por lo que podemos concluir que nuestro por el momento nuestro modelo presenta sobreajuste.**

## Conclusión General

Empezamos nuestro proceso importando el dataset con los comportamiento de los usuarios. Luego segmentamos nuestro dataset en conjunto de entranamiento, validación y de prueba, con proporción 60% para el conjutno de entrenamiento y 20% para el validación y de prueba.

Probamos tres modelos: árbol de decisión, bosque aleatorio y regresión logística. Se iteraron sus hiperparámetros y se determinó que el mejor modelo para el proyecto es el de bosque aleatorio.

Por último, probamos la precisión de nuestro conjunto de validación frente al de prueba utilizando escogido.

**1. El modelo de bosque aleatorio con una cantidad de estimadores de 40 (n_estimators) y profundida de árboles de 8 genera la mayor precisión, 0.795**

Se iteró el modelo probando diferente número de estimadores y profundidad de árboles, concluyendo que 23 estimadores es el mejor para el modelo.

**2. El modelo de bosque aleatorio muestra sobreajuste al utilizarlo con el conjunto de prueba**

Comparamos la exactitud del modelo tanto en el conjunto de entrenamiento como en el conjunto de prueba, encontrando diferencias entre ambos. Esto sugiere que la robustez del modelo no es muy alta y que hay cierto grado de sobreajuste.